# Training

In [ ]:
from treecat_exp.util import load_object
from matplotlib import pyplot
import numpy as np
import torch
from pyro.contrib.tabular.treecat import print_tree
%matplotlib inline
%config InlineBackend.rc = {'figure.facecolor': (1, 1, 1, 1)}
# %config InlineBackend.figure_format = 'svg'

To perform training, run `python train.py`

In [ ]:
DATASET = 'credit.treecat.8'
meta = load_object('results/train/{}.meta.pkl'.format(DATASET))
model = load_object('results/train/{}.model.pkl'.format(DATASET))
print(list(meta.keys()))
args = meta['args']
stepsizes = meta['stepsizes']

In [ ]:
pyplot.figure(figsize=(9, 6))
pyplot.plot(meta['losses'], label='loss')
pyplot.ylabel('ELBO Loss  (nats / cell)')
pyplot.xlabel('SVI Step')
asymptote = np.mean(meta['losses'][int(0.9 * len(meta['losses'])):])
pyplot.axhline(asymptote, color='orange', linestyle='--', lw=1.5,
               label='{:0.3g}'.format(asymptote))
pyplot.yscale('log')
pyplot.xlim(0, len(meta["losses"]))
pyplot.title('Training on {} data (m={}, lr={}, ar={}, b={})'.format(
             args.dataset, args.capacity, args.learning_rate,
             args.annealing_rate, args.batch_size))
pyplot.legend(loc='best')
pyplot.tight_layout()

In [ ]:
def downsample(series, factor):
    series = torch.tensor(series)
    series = series[:len(series) // factor * factor]
    return series.reshape(-1, factor).mean(-1).numpy()

In [ ]:
factor = (1 + len(stepsizes) // 100)
pyplot.figure(figsize=(9, 6))
for key in stepsizes[-1]:
    series = downsample([s.get(key, 0) for s in stepsizes], factor)
    X = range(0, len(series) * factor, factor)
    if key == 'tree':
        pyplot.plot(X, series, 'k--', label="tree", lw=1)
    else:
        pyplot.plot(X, series, alpha=0.5, lw=0.5)
pyplot.ylabel('Stepsize')
pyplot.xlabel('SVI Step')
pyplot.xlim(0, len(meta["losses"]))
pyplot.ylim(1e-4, 1e1)
pyplot.yscale('log')
pyplot.title('Parameter steps on {} data (m={}, lr={}, ar={}, b={})'.format(
             args.dataset, args.capacity, args.learning_rate,
             args.annealing_rate, args.batch_size))
pyplot.legend(loc='best')
pyplot.tight_layout()

In [ ]:
from IPython.display import HTML
tree = print_tree(model.edges, [f.name for f in model.features])
HTML('<pre><p style="font-size: 10pt; line-height: 11pt;">{}</p></pre>'.format(tree))